In [1]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!pip install git+https://github.com/afnan47/cuda.git

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-hahe2ywu
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-hahe2ywu
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=792bcc632ca29cbff4abf4c3d65c2b4b96ef80c61fe095e2a2f8c67b9b2ba696
  Stored in directory: /tmp/pip-ephem-wheel-cache-swuxjq15/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [5]:
%%writefile bfs.cu

#include <iostream>
#include <vector>
#include <queue>
#include <ctime>
#include <cuda_runtime.h>

using namespace std;

int main()
{
    int num_vertices, num_edges, source;
    cout << "Enter number of vertices: ";
    cin >> num_vertices;
    cout << "Enter number of edges: ";
    cin >> num_edges;
    cout << "Enter source node: ";
    cin >> source;

    // Input validation
    if (source < 1 || source > num_vertices)
    {
        cout << "Invalid source node!" << endl;
        return 1;
    }

    vector<vector<int>> adj_list(num_vertices + 1);
    cout << "Enter the edges:" << endl;
    for (int i = 0; i < num_edges; i++)
    {
        int u, v;
        cin >> u >> v;
        // Input validation for edges
        if (u < 1 || u > num_vertices || v < 1 || v > num_vertices)
        {
            cout << "Invalid edge: " << u << " " << v << endl;
            return 1;
        }
        adj_list[u].push_back(v);
        adj_list[v].push_back(u);
    }

    queue<int> q;
    vector<bool> visited(num_vertices + 1, false);
    q.push(source);
    visited[source] = true;

    // CUDA event creation for measuring execution time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    while (!q.empty())
    {
        int curr_vertex = q.front();
        q.pop();
        cout << curr_vertex << " ";

        // Sequential loop for neighbors
        for (int i = 0; i < adj_list[curr_vertex].size(); i++)
        {
            int neighbour = adj_list[curr_vertex][i];
            if (!visited[neighbour])
            {
                visited[neighbour] = true;
                q.push(neighbour);
            }
        }
    }

    // Record and calculate execution time
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    cout << endl << "Execution Time: " << milliseconds << " ms" << endl;

    // Cleanup
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}


Writing bfs.cu


In [6]:

!nvcc bfs.cu -o bfs

In [7]:
!./bfs

Enter number of vertices: 5
Enter number of edges: 4
Enter source node: 1
Enter the edges:
1 2
1 3
2 4
3 5
1 2 3 4 5 
Execution Time: 0.0184 ms
